In [12]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from scripts.all_imports import *
from scripts.utils import  CONFIG, FILE_PATHS, PATH_ROOT, INPUT_PATH, OUTPUT_PATH

# Acessa as variáveis globais
print("Configuration:", CONFIG)
print("Path:", FILE_PATHS)
print("Source:", PATH_ROOT)
print("Input:",INPUT_PATH)
print("Output:",OUTPUT_PATH)
print("File",CONFIG["files"]["file_data_pubmed"])


Configuration: {'api_key': '09921aadccbafb8e5913ae1c99e2fafbe00a', 'api_email': 'leandrorodrigues.s@gmail.com', 'config': {'search_type': 1, 'time_sleep': '0.1', 'file_save_periodically': 10, 'save_xml': True, 'search': 0.1, 'search_article_one': True}, 'directories': {'output': './data/processed', 'xml': './data/processed/xml', 'input': './data/input', 'logs': './logs', 'type_files': './data/processed/files_type'}, 'files': {'file_data_error': './metadata/NotProcessed.csv', 'file_data_input': 'References_MJFF_15122024.csv', 'file_parsing_pubmed': './metadata/PubMed_Metadata.csv', 'file_parsing_bibliometrix': './metadata/Bibliometrix_Metadata.csv', 'file_data_pubmed': './files_type/PubMed_format.txt', 'file_bibliometrix': './files_type/Bibliometrix_format.xlsx', 'file_vsviewer': 'VosViewer.ris', 'file_bibtex': 'bibtex.bib', 'file_web_of_science': 'web_of_science.txt'}, 'logging': {'log_file': './logs/execution.log', 'level': 'INFO', 'format': '%(asctime)s - %(levelname)s - %(message)s'

In [13]:
# deletes all processing files and execution logs, with the option to comment out the lines if you don't want them executed:
clear_all_processec()

File c:\Fontes\pubmed-bibliometric-parser\data\processed\./metadata/NotProcessed.csv not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\References_MJFF_15122024.csv not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\./metadata/PubMed_Metadata.csv not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\./metadata/Bibliometrix_Metadata.csv not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\./files_type/PubMed_format.txt not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\./files_type/Bibliometrix_format.xlsx not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\VosViewer.ris not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\bibtex.bib not found.
File c:\Fontes\pubmed-bibliometric-parser\data\processed\web_of_science.txt not found.


In [14]:
logging.info("Application started.")
print(f" paths: Input -> {INPUT_PATH} \n output -> {OUTPUT_PATH}")
df_pubmed = pd.DataFrame()
    
logging.info(f"Output file paths: {FILE_PATHS}")
logging.info(f"Output config: {CONFIG}")

 paths: Input -> c:\Fontes\pubmed-bibliometric-parser\data\input 
 output -> c:\Fontes\pubmed-bibliometric-parser\data\processed


In [15]:
# Define the file path for the input data. If the key "file_data_input" exists in the configuration,
# its value will be used. Otherwise, the default file name "InputSearch.csv" is used.
# Usage of the function

FILE_INPUT_SEARCH = os.path.join(INPUT_PATH, CONFIG["files"].get("file_data_input", "InputSearchDoi.csv"))
print(f"Search file: {FILE_INPUT_SEARCH}")

logging.info(f"Search file in: {FILE_INPUT_SEARCH}")

pd.set_option('display.max_columns', None)
df_search = load_csv_to_dataframe(FILE_INPUT_SEARCH, '|')
#print(df_search)
df_search.head(2)


Search file: c:\Fontes\pubmed-bibliometric-parser\data\input\References_MJFF_15122024.csv


,title,doi
0,Bidirectional relationship between olfaction a...,10.1038/s41531-024-00838-4
1,Slovak database of speech affected by neurodeg...,10.1038/s41597-024-04171-6


In [16]:
import pandas as pd

# Remover valores NaN na coluna 'doi' antes de verificar duplicatas
df_search_clean = df_search.dropna(subset=['doi'])

# Verificar duplicatas na coluna 'doi'
duplicated_dois = df_search_clean[df_search_clean['doi'].duplicated(keep=False)]

# DataFrame com DOIs duplicados
df_duplicated_dois = duplicated_dois[['doi']].drop_duplicates()
df_duplicated_dois['count'] = duplicated_dois['doi'].value_counts().loc[df_duplicated_dois['doi']].values

# Verificar duplicatas na coluna 'title'
duplicated_titles = df_search_clean[df_search_clean['title'].duplicated(keep=False)]

# DataFrame com títulos duplicados
df_duplicated_titles = duplicated_titles[['title']].drop_duplicates()
df_duplicated_titles['count'] = duplicated_titles['title'].value_counts().loc[df_duplicated_titles['title']].values

# Verificar duplicatas na combinação 'title' + 'doi'
duplicated_combinations = df_search_clean[df_search_clean.duplicated(subset=['title', 'doi'], keep=False)]

# DataFrame com combinações duplicadas
df_duplicated_combinations = (
    duplicated_combinations[['title', 'doi']]
    .value_counts()
    .reset_index()
    .rename(columns={0: 'count'})
)

# Exibir os resultados no console
print("DOIs Duplicados:")
print(df_duplicated_dois)

print("\nTítulos Duplicados:")
print(df_duplicated_titles)

print("\nCombinações Duplicadas (Título + DOI):")
print(df_duplicated_combinations)

FILE_INPUT_SEARCH = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_pubmed", "PubMed_Metadata.csv"))

OUTPUT_PATH
# (Opcional) Salvar os resultados em arquivos CSV
output_file = os.path.join(OUTPUT_PATH, "duplicated_dois.csv")
df_duplicated_dois.to_csv(output_file, index=False)

output_file = os.path.join(OUTPUT_PATH, "duplicated_titles.csv")
df_duplicated_titles.to_csv(output_file, index=False)

output_file = os.path.join(OUTPUT_PATH, "duplicated_combinations.csv")
df_duplicated_combinations.to_csv(output_file, index=False)


DOIs Duplicados:
Empty DataFrame
Columns: [doi, count]
Index: []

Títulos Duplicados:
                                                  title  count
33    Brain-derived and in vitro-seeded alpha-synucl...      2
414   Downregulation of Dickkopf-3, a Wnt antagonist...      2
531   Scaling of an antibody validation procedure en...      2
619   Journal Production Guidance for Software and D...      2
668   A Fixable Fluorescence‐Quenched Substrate for ...      2
890   Novel therapeutic approaches to target neurode...      2
925   Differences in Sex‐Specific Frequency of Gluco...      2
1090  A Collaboration for Guiding the Rational Desig...      2
1245  Decreased Water Mobility Contributes To Increa...      2
1342                                     Issue Overview      2
1434                                          Abstracts      5
1626  An MDS Evidence‐Based Review on Treatments for...      2
2166                            Editorial introductions      2
2407  High diagnostic performanc

In [6]:
# Function to load and process articles based on their DOI and, if necessary, their title.
def process_query(i, df_articles):
    """
    Queries article information using the DOI found in df_articles. If no results 
    are returned for the DOI, it attempts a fallback search by title.

    Parameters
    ----------
    i : int
        The index of the current row in df_articles being processed.
    df_articles : pd.DataFrame
        A DataFrame containing article data. It should have at least:
        - 'doi': The DOI of the article.
        - 'title': The title of the article.

    Returns
    -------
    tuple (count, id_list)
        count : int
            The number of results retrieved from the query.
        id_list : list or None
            A list of IDs returned by the query. None if no results are found.

    Workflow
    --------
    1. Construct a query string using the article's DOI and attempt to retrieve results.
    2. If no results are found, attempt a second query using the article's title.
    3. Log the queries and their results.
    4. Return the number of results (count) and the list of IDs (id_list).

    Error Handling
    --------------
    - In case of an error, log the exception and return (0, None).

    Notes
    -----
    - The function assumes that `request_count` is defined elsewhere, which should handle 
      the actual querying process and return (count, id_list).
    - Logging is used for debugging and tracing query execution steps.
    """
    try:        
        # Construct and log the initial DOI-based query
        query_search = f"{df_articles.loc[i, 'doi']}[DOI]"
        logging.info(f"Search query: {query_search}")

        # Execute the query and retrieve count and IDs
        count, id_list = request_count(query_search)
        
        logging.info(f"{i} - Number of results: {count} - Searching by DOI: {df_articles.loc[i, 'doi']}")
        
        # If no results found by DOI, try using the title
        if count == 0:
            query_search = df_articles.loc[i, "title"]
            count, id_list = request_count(query_search)
            logging.info(f"{i} - Number of results: {count} - Searching by title: {query_search}")

        return count, id_list
    
    except Exception as e:
        # Log and print errors, returning (0, None) if any exception occurs
        print(f"Error at row {i}: {e}")
        logging.error(f"Error at row {i}: {e}")
        return 0, None


In [7]:
from tqdm import tqdm
import logging
import time
import pandas as pd

def process_all_articles(df_search, PUBMED_FILE_OUTPUT, BIBLIOMETRIX_FILE_OUTPUT):
    """
    Process a list of articles from a DataFrame, retrieve PubMed data, parse it into 
    two different formats (bibliometrix and pubmed), and periodically save the results.

    Parameters
    ----------
    df_search : pd.DataFrame
        Input DataFrame containing the articles to be processed. Expected columns include:
        - 'doi': The article's DOI
        - 'title': The article's title
    
    PUBMED_FILE_OUTPUT : str
        The path and filename for saving PubMed-formatted results.
    
    BIBLIOMETRIX_FILE_OUTPUT : str
        The path and filename for saving Bibliometrix-formatted results.

    Workflow
    --------
    1. Determine the total number of articles to process from df_search.
    2. Use tqdm to create a progress bar for tracking processing status.
    3. For each article (row in df_search):
       a) Pause for a configured duration to avoid rate limits (from CONFIG).
       b) Run process_query to get the number of matching articles (count) and their IDs (id_list) based on DOI or title.
       c) If the configuration allows only one result, set total_retorno = 1; otherwise, allow multiple results (infinite).
       d) If the number of returned articles matches the expected count:
          - For each returned article_id:
            i. Update the progress bar with the current article's status.
            ii. Log the processing of the current PubMed ID and related metadata.
            iii. Request XML data from PubMed (using request_data).
            iv. If configured, save the raw XML data (save_xml_data).
            v. Parse the XML into bibliometrix and pubmed DataFrames (parse_xml_to_bibliometrix_df and parse_xml_to_pubmed_df).
            vi. Concatenate these new rows into the accumulating DataFrames (df_bibliometrix and df_pubmed).
            vii. Update the progress bar for each processed article.
       e) If no results match the query, log that the query failed.

    4. Periodically (based on CONFIG['config']['file_save_periodically']) or at the end:
       - Save the bibliometrix and pubmed DataFrames to their respective output files.
       - Replace newline characters, log the action, update the progress bar status, and clear the DataFrames.

    5. After processing all articles, print a completion message.

    Notes
    -----
    - The CONFIG object, process_query, request_data, parse_xml_to_bibliometrix_df, parse_xml_to_pubmed_df,
      save_xml_data, and save_data_to_file functions are assumed to be defined elsewhere.
    - The DataFrames df_pubmed and df_bibliometrix are dynamically built and reset after periodic saves to manage memory usage.
    - Logging and tqdm progress updates provide transparency into the ongoing process.

    Example
    -------
    process_all_articles(df_input, 'pubmed_output.csv', 'bibliometrix_output.csv')
    """

    total_articles = len(df_search)
    count = 0
    df_pubmed = None
    df_bibliometrix = None
    
    # Create a progress bar for processing articles
    with tqdm(total=total_articles, desc="Processing PubMed articles", unit="file") as pbar:
        # Main processing loop over each article
        for i, row in df_search.iterrows():
            # Pause to avoid hitting rate limits (configured via CONFIG)
            time.sleep(float(CONFIG['config']['time_sleep']))
            
            # Query the article based on DOI or title
            count, id_list = process_query(i, df_search)
            doi = df_search.loc[i, 'doi']

            # Determine whether to accept only one result or multiple
            if CONFIG['config']['search_article_one']:
                total_retorno = 1
            else:
                total_retorno = float('inf')  # No limit on the number of results
            
            # If the number of returned articles matches the expected count
            if count == total_retorno:
                try:
                    #for idx, article_id in enumerate(id_list, start=1):
                    for article_id in set(id_list):  # Evita duplicação nos IDs retornados

                        # Update progress bar with current article information
                        pbar.set_postfix({"Current PubMed ID": article_id,
                                          "Title": df_search.loc[i, 'title'][:30] + "..."})
                        
                        # Log processing details
                        logging.info(f"{i} - {count} Processing PubMed ID: {article_id} - DOI: {doi} - title: {df_search.loc[i, 'title']}")

                        # Request XML data for the current article
                        xml_data = request_data(article_id)

                        # If configured, save the raw XML data
                        if bool(CONFIG['config']['save_xml']):
                            save_xml_data(article_id, xml_data)

                        # Parse the XML into bibliometrix and pubmed DataFrames
                        new_data_bibliometrix = parse_xml_to_bibliometrix_df(xml_data)
                        new_data_pubmed = parse_xml_to_pubmed_df(xml_data)

                        # Concatenate new rows to the accumulating DataFrames
                        df_bibliometrix = pd.concat([df_bibliometrix, new_data_bibliometrix], ignore_index=True)
                        df_pubmed = pd.concat([df_pubmed, new_data_pubmed], ignore_index=True)

                        # Update the progress bar after processing each article
                        pbar.update(1)

                except Exception as e:
                    logging.error(f"Error processing row {i}: {e}")
            else:
                # Log if the query failed or returned no results
                logging.info({"error": "Query failed", "query": id_list})

            # Periodically save bibliometrix data according to configuration or at the last record
            if (i + 1) % int(CONFIG['config']['file_save_periodically']) == 0 or i == len(df_search) - 1:
                if df_bibliometrix is not None and not df_bibliometrix.empty:
                    df_bibliometrix = df_bibliometrix.replace(r'\n', ' ', regex=True)

                    # Save bibliometrix DataFrame to file
                    save_data_to_file(df_bibliometrix, BIBLIOMETRIX_FILE_OUTPUT)
                    logging.info("Saving Bibliometrix file.")
                    pbar.set_postfix({"Status": "Saving Bibliometrix file"})
                    df_bibliometrix = df_bibliometrix.iloc[0:0]

            # Periodically save pubmed data according to configuration or at the last record
            if (i + 1) % int(CONFIG['config']['file_save_periodically']) == 0 or i == len(df_search) - 1:
                if df_pubmed is not None and not df_pubmed.empty:
                    df_pubmed = df_pubmed.replace(r'\n', ' ', regex=True)

                    # Save pubmed DataFrame to file
                    save_data_to_file(df_pubmed, PUBMED_FILE_OUTPUT)
                    logging.info("Saving PubMed file.")
                    pbar.set_postfix({"Status": "Saving PubMed file"})
                    df_pubmed = df_pubmed.iloc[0:0]


    print("\nProcessing completed!")


In [ ]:
# Define output file paths for the parsed PubMed and Bibliometrix data. 
# The file names are determined by checking the CONFIG dictionary:
# - If "file_parsing_pubmed" exists, use its value, otherwise use "PubMed_Metadata.csv".
# - If "file_parsing_bibliometrix" exists, use its value, otherwise use "Bibliometrix_Metadata.csv".
#
# These files will store the results processed by the `process_all_articles` function.

PUBMED_FILE_OUTPUT = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_pubmed", "PubMed_Metadata.csv"))
BIBLIOMETRIX_FILE_OUTPUT = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_bibliometrix", "Bibliometrix_Metadata.csv"))

# Run the main processing function with the input DataFrame `df_search` and the defined output paths.
process_all_articles(df_search, PUBMED_FILE_OUTPUT, BIBLIOMETRIX_FILE_OUTPUT)


In [ ]:
# Define the input file path for reading processed PubMed metadata.
# The file name is determined by checking the CONFIG dictionary:
# - If "file_parsing_pubmed" exists under CONFIG["files"], use that value.
# - Otherwise, default to "PubMed_Metadata.csv".
FILE_INPUT_SEARCH = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_pubmed", "PubMed_Metadata.csv"))

# Print and log the path of the file being read.
print(f"Search file: {FILE_INPUT_SEARCH}")
logging.info(f"Search file in: {FILE_INPUT_SEARCH}")

# Configure pandas to display all columns when showing DataFrame outputs.
pd.set_option('display.max_columns', None)

# Load the CSV file into a DataFrame using the '|' delimiter.
# The DataFrame `df_pubmed` now contains the processed PubMed metadata.
df_pubmed = load_csv_to_dataframe(FILE_INPUT_SEARCH, '|')

# Print the number of rows in `df_search` for comparison or reference.
print(len(df_search))

# Display the first two rows of `df_pubmed` to quickly verify the loaded data.
df_pubmed.head(2)


In [ ]:
# Ensure that the DOIs are properly formatted by:
# 1. Replacing any NaN values with empty strings.
# 2. Removing leading and trailing spaces.
# 3. Converting all characters to uppercase.
df_search['doi'] = df_search['doi'].fillna('').str.strip().str.upper()
df_pubmed['ArticleIdListDOI'] = df_pubmed['ArticleIdListDOI'].fillna('').str.strip().str.upper()

# Create sets of DOIs from both DataFrames to efficiently compare their contents.
doi_search = set(df_search['doi'])
doi_pubmed = set(df_pubmed['ArticleIdListDOI'])

# Identify which DOIs are present in df_search but not in df_pubmed.
doi_difference = doi_search - doi_pubmed

# Filter df_search to retrieve only the rows corresponding to these missing DOIs.
missing_dois = df_search[df_search['doi'].isin(doi_difference)]

# Count how many DOIs are missing.
num_differences = len(missing_dois)
print(f"Number of missing DOIs: {num_differences}")

# Determine the output file path for the missing references file.
# If "file_data_error" is specified in CONFIG["files"], use it; otherwise, use "error.csv".
REFERENCIAS_INEXISTENTES = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_data_error", "NotProcessed.csv"))


# Save the missing DOIs to the specified output file.
save_data_to_file(missing_dois, REFERENCIAS_INEXISTENTES)

# Note: The commented-out code below shows a direct approach to saving the missing DOIs as a CSV file.
# missing_dois.to_csv('missing_dois.csv', index=False, encoding='utf-8')
# print("File 'missing_dois.csv' saved successfully!")
